In [3]:
# predict using autogluon
import os
import sys
import pandas as pd
import numpy
from autogluon.tabular import TabularPredictor

d:\PJATK\Dyplom\automated_flat_3D_models_generator\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
cropped_images_features = pd.read_parquet('./cropped_images_features.parquet')
not_cropped_images_features = pd.read_parquet('./not_cropped_images_features.parquet')
# combine the two dataframes
df = pd.concat([cropped_images_features, not_cropped_images_features])
# mix the rows
df = df.sample(frac=1).reset_index(drop=True)

In [8]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=True)

In [9]:
# train model
label = 'label'
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20241216_091549"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.0
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          16
Memory Avail:       4.62 GB / 15.37 GB (30.1%)
Disk Space Avail:   189.84 GB / 931.51 GB (20.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         

In [10]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetTorch,0.963410,0.960396,accuracy,0.143354,0.065512,150.930152,0.143354,0.065512,150.930152,1,True,7
1,WeightedEnsemble_L2,0.962656,0.961339,accuracy,1.588685,0.665930,151.578706,0.001584,0.001015,0.069609,2,True,8
2,RandomForestEntr,0.796680,0.803395,accuracy,0.201182,0.084921,24.639985,0.201182,0.084921,24.639985,1,True,4
3,RandomForestGini,0.784232,0.800566,accuracy,0.219850,0.085470,17.925622,0.219850,0.085470,17.925622,1,True,3
4,ExtraTreesEntr,0.762354,0.780764,accuracy,0.232606,0.089882,5.316005,0.232606,0.089882,5.316005,1,True,6
5,ExtraTreesGini,0.759902,0.768505,accuracy,0.275814,0.090147,5.301010,0.275814,0.090147,5.301010,1,True,5
6,KNeighborsUnif,0.705960,0.725601,accuracy,1.492766,0.614019,0.579883,1.492766,0.614019,0.579883,1,True,1
7,KNeighborsDist,0.664655,0.683168,accuracy,1.443747,0.599403,0.578945,1.443747,0.599403,0.578945,1,True,2


In [19]:
# get hyperparameters of the best model
predictor.info()['model_info']['NeuralNetTorch']['hyperparameters']

{'num_epochs': 1000,
 'epochs_wo_improve': None,
 'activation': 'relu',
 'embedding_size_factor': 1.0,
 'embed_exponent': 0.56,
 'max_embedding_dim': 100,
 'y_range': None,
 'y_range_extend': 0.05,
 'dropout_prob': 0.1,
 'optimizer': 'adam',
 'learning_rate': 0.0003,
 'weight_decay': 1e-06,
 'proc.embed_min_categories': 4,
 'proc.impute_strategy': 'median',
 'proc.max_category_levels': 100,
 'proc.skew_threshold': 0.99,
 'use_ngram_features': False,
 'num_layers': 4,
 'hidden_size': 128,
 'max_batch_size': 512,
 'use_batchnorm': False,
 'loss_function': 'auto'}